In [1]:
# Declaration

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

In [2]:
def binstr_to_name(str):
    d = ['INSULIN', 'METFORMIN', 'GLP1', 'SGLT2']
    result_arr = []
    result_str = ''
    for i in range(0, len(str)):
        if str[i] == '1':
            result_arr.append(d[i])
    if result_arr:
        result_str = ' + '.join(result_arr)
    return result_str

# Drugs used for first-time medications

Observe the use of drugs used in the first treatment, either single or combination
This observation is based on HBA1c level refers to NICE guideline

| CLASS | DRUGS                              | COUNT    | COUNT(%) | HBA1C(%) |
| ----- | ---------------------------------- | -------- | -------- | -------- |
| 1101  | INSULIN + METFORMIN + SGLT2        | 1        | 0.03     | 9.10     |
| 1000  | INSULIN                            | 174      | 5.93     | 6.97     |
| 0111  | METFORMIN + GLP1 + SGLT2           | 3        | 0.10     | 9.83     |
| 1100  | INSULIN + METFORMIN				 | 44       | 1.50     | 7.25     |
| 0110  | METFORMIN + GLP1                   | 774      | 26.36    | 10.06    |
| 0101  | METFORMIN + SGLT2                  | 1.       | 0.03     | 9.40     |
| 0100  | METFORMIN                          | 1939     | 66.04    | 8.03     |

-  Metformin is the most used as first time medication for diabetes with 66% of times

In [3]:
data = pd.read_csv('diabetes/p_m_o.csv', converters={'MEDCLASS': lambda x: str(x)})
data = data[data['AGEMEDICATION'] == data['AGEFIRSTMEDICATION']]

classlist = list(set(data['MEDCLASS']))
result = pd.DataFrame(data=[], index=classlist, columns=['DRUGS', 'COUNT', 'COUNT(%)', 'HBA1C(%)', 'BMI(kg/m2)'])

num_of_firstmed = len(data)
for c in classlist:
    temp = data[data['MEDCLASS'] == c]
    result.at[c, 'DRUGS'] = binstr_to_name(c)
    result.at[c, 'COUNT'] = len(temp['HBA1C'])
    result.at[c, 'COUNT(%)'] = '%0.2f' % ((len(temp)/num_of_firstmed)*100)
    result.at[c, 'HBA1C(%)'] = '%0.2f' % temp['HBA1C'].median()
    result.at[c, 'BMI(kg/m2)'] = '%0.2f' % temp['BMI'].median()


result

,DRUGS,COUNT,COUNT(%),HBA1C(%),BMI(kg/m2)
1101,INSULIN + METFORMIN + SGLT2,1,0.01,9.10,44.34
1100,INSULIN + METFORMIN,116,1.63,7.60,30.98
0110,METFORMIN + GLP1,1888,26.51,9.80,39.36
0100,METFORMIN,4668,65.55,7.90,31.41
1000,INSULIN,439,6.16,6.90,27.72
0111,METFORMIN + GLP1 + SGLT2,6,0.08,10.00,48.89
0101,METFORMIN + SGLT2,3,0.04,9.30,45.01


# Drugs used in overall medications

In overall, The most used medication for diabetes is a combination of Metformin + GLP1 (29.64%)

| CLASS | DRUGS                              | COUNT    | COUNT(%) | HBA1C(%) |
| ----- | ---------------------------------- | -------- | -------- | -------- |
| 0100  | METFORMIN                          | 1939     | 26.79    | 8.03     |
| 1000  | INSULIN                            | 246      | 3.40     | 6.65     |
| 1100  | INSULIN + METFORMIN				 | 1148     | 15.86    | 7.23     |
| 0110  | METFORMIN + GLP1                   | 2145     | 29.64    | 9.58     |
| 1110  | INSULIN + METFORMIN + GLP1         | 280      | 3.87     | 7.87     |
| 1111  | INSULIN + METFORMIN + GLP1 + SGLT2 | 168      | 2.32     | 8.06     |
| 0111  | METFORMIN + GLP1 + SGLT2           | 1311     | 18.12    | 9.67     |


In [5]:
data = pd.read_csv('diabetes/p_m_o.csv', converters={'MEDCLASS': lambda x: str(x)})

classlist = list(set(data['MEDCLASS']))
result = pd.DataFrame(data=[], index=classlist, columns=['DRUGS', 'COUNT', 'COUNT(%)', 'HBA1C(%)', 'BMI(kg/m2)'])

for c in classlist:
    temp = data[data['MEDCLASS'] == c ]
    result.at[c, 'DRUGS'] = binstr_to_name(c)
    result.at[c, 'COUNT'] = len(temp['HBA1C'])
    result.at[c, 'COUNT(%)'] = '%0.2f' % ((len(temp)/len(data))*100)
    result.at[c, 'HBA1C(%)'] = '%0.2f' % temp['HBA1C'].median()
    result.at[c, 'BMI(kg/m2)'] = '%0.2f' % temp['BMI'].median()

result

,DRUGS,COUNT,COUNT(%),HBA1C(%),BMI(kg/m2)
1101,INSULIN + METFORMIN + SGLT2,83,0.48,9.90,47.58
1111,INSULIN + METFORMIN + GLP1 + SGLT2,28,0.16,9.25,44.98
1100,INSULIN + METFORMIN,3582,20.82,7.20,37.52
1110,INSULIN + METFORMIN + GLP1,111,0.65,9.30,43.52
0110,METFORMIN + GLP1,5916,34.39,9.40,42.95
0100,METFORMIN,4668,27.13,7.90,31.41
1000,INSULIN,608,3.53,6.60,28.77
0111,METFORMIN + GLP1 + SGLT2,1345,7.82,10.00,48.53
0101,METFORMIN + SGLT2,864,5.02,9.40,45.46


# Medication efficacy

Medication efficacy will be measured by how long the medication is given to the patient until the intensification needed.

In [12]:
data = pd.read_csv('diabetes/m.csv', converters={'MEDCLASS': lambda x: str(x)})
data.drop(['ENCOUNTER', 'MEDNUM', 'PREVMED', 'PREVMEDSTART', 'PREVMEDDUR'], axis=1, inplace=True)
data = data.set_index(['PATIENT', 'MEDSTART']).sort_index().reset_index()
data['MEDSTART'] = pd.to_datetime(data['MEDSTART'], format='%Y-%m-%d')
data['INDUR'] = 0
data['INTYPE'] = 0
data['INCLASS'] = '0000'

# menghitung durasi pemakaian obat sampai terjadi intensifikasi

# 0:patient 1:medstart 2:insulin 3:metformin 4:glp1 5:sglt2 6:combination 7:medclass 8:indur 9:intype 10:inclass
arr = data.values
patient =  ''
for i in range(0, len(arr)):
    if arr[i][0] != patient: # new patient
        index = i
        patient = arr[i][0]
        combination = arr[i][6]
    else: # same patient
        if arr[i][6] != combination: # intensify medication
            # update the intdur of row at index
            arr[index][8] = (arr[i][1] - arr[index][1]).days
            arr[index][10] = arr[i][7]

            if combination < arr[i][6]:
                arr[index][9]  =  1
            else:
                arr[index][9]  =  -1
                
            # set current state
            combination = arr[i][6]
            index = i
    i += 1

arr = pd.DataFrame(data=arr, columns=['PATIENT', 'MEDSTART', 'INSULIN', 'METFORMIN', 'GLP1', 'SGLT2', 'COMBINATION', 'MEDCLASS', 'INDUR', 'INTYPE', 'INCLASS'])
arr = arr[(arr['INDUR'] > 0) & (arr['INTYPE']  == 1)]

display(arr)
# calculate intensification duration for each class
classes = set(list(arr['MEDCLASS']))
for c in classes:
    print('class : ' + binstr_to_name(c))
    print('data : '  + str(len(arr[arr['MEDCLASS'] == c])))
    print('duration to intensification : '  + str(arr[arr['MEDCLASS'] == c]['INDUR'].median()) + '(days)  /  ' + 
         str('%0.2f' % (arr[arr['MEDCLASS'] == c]['INDUR'].median()/365)) + '(years)')
    print('')

,PATIENT,MEDSTART,INSULIN,METFORMIN,GLP1,SGLT2,COMBINATION,MEDCLASS,INDUR,INTYPE,INCLASS
0,000177c6-f76b-432b-9493-5a88bc9fb6bd,1957-02-27,0,1,1,0,2,0110,14639,1,1110
3,00018e41-aba7-4cb3-aeed-fd2d7bcbc6a7,1964-07-10,0,1,0,0,1,0100,2040,1,1100
5,00026c2e-b6f6-4afd-a6f2-10f5b3c609d6,2006-05-22,0,1,0,0,1,0100,3335,1,0110
7,0003b74a-1650-480e-bbcc-7ec5c0594299,1979-03-07,0,1,0,0,1,0100,2765,1,1100
10,0008b1a6-e0d6-413e-a015-5b65f92cb619,2013-01-09,0,1,1,0,2,0110,351,1,0111
...,...,...,...,...,...,...,...,...,...,...,...
91227,fff8017e-e0c3-42f6-92f0-7362387b66b5,1978-08-17,0,1,0,0,1,0100,3168,1,1100
91229,fff8a535-84ad-4d27-8311-3e96a648f779,1991-04-05,0,1,0,0,1,0100,1524,1,1100
91235,fffc1f57-f025-459f-ad1a-757698cf313c,1996-02-17,0,1,0,0,1,0100,2587,1,1100
91240,ffff216a-4273-4843-b9f3-9f81ace31098,1970-12-31,0,1,0,0,1,0100,1970,1,1100


class : INSULIN + METFORMIN + GLP1
data : 2
duration to intensification : 5.5(days)  /  0.02(years)

class : METFORMIN + GLP1
data : 1552
duration to intensification : 3914.5(days)  /  10.72(years)

class : METFORMIN
data : 18707
duration to intensification : 4014.0(days)  /  11.00(years)

class : INSULIN
data : 1
duration to intensification : 4.0(days)  /  0.01(years)

class : METFORMIN + GLP1 + SGLT2
data : 15
duration to intensification : 1523.0(days)  /  4.17(years)



## [Learning]

### Goal 1 : Predicting medication for new patient

There is a significant difference when the number of medication combination is included in the input data. The problem is, how to define the usage of intensification.

Previous medication and previous medication duration also have a significant impact on the prediction accuracy



In [ ]:
data = pd.read_csv('diabetes/p_m_o.csv', converters={'MEDCLASS': lambda x: str(x), 'PREVMED': lambda x: str(x)})

data = data.drop(
    ['PATIENT', 'ENCOUNTER', 'MEDSTART', 'INSULIN', 'METFORMIN', 'GLP1', 'SGLT2', 
     'DEAD', 'AGEDEAD', 'DAYSLIVEFIRSTMED', 'OBSDATE', 'PREVMEDSTART']
    , axis=1, errors='ignore')

data['GENDER'] = data['GENDER'].replace({"F":0, "M":1})

data = pd.get_dummies(data = data, columns=['RACE', 'ETHNICITY'])

# convert binary string to decimal
data['MEDCLASS'] = data['MEDCLASS'].apply(lambda x: int(x, 2))
data['PREVMED'] = data['PREVMED'].apply(lambda x: int(x, 2))

data.head(10)

In [ ]:
data.corr()['MEDCLASS'].sort_values()

In [ ]:
y = data['MEDCLASS']
x = data.drop(['MEDCLASS'], axis=1)

# data split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, stratify=y, random_state=112)

### Training

In [ ]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)

logreg = LogisticRegression(penalty="l2", class_weight='balanced', max_iter=10000)

logreg.fit(x_train, y_train)

y_hat_train = logreg.predict(x_train)

#display(accuracy_score(y_train, y_hat_train))
scores = cross_val_score(logreg, x_train, y_train, cv=4)
display(scores.mean())

### Testing

In [ ]:
x_test = scaler.transform(x_test)

y_hat_test = logreg.predict(x_test)

#display(accuracy_score(y_test, y_hat_test))
scores = cross_val_score(logreg, x_test, y_test, cv=4)
display(scores.mean())

In [ ]:
def binstr_to_name(str):
    d = ['INSULIN', 'METFORMIN', 'GLP1', 'SGLT2']
    result_arr = []
    result_str = ''
    for i in range(0, len(str)):
        if str[i] == '1':
            result_arr.append(d[i])
    if result_arr:
        result_str = ' + '.join(result_arr)
    return result_str

# confusion matrix
conmat = confusion_matrix(y_test, y_hat_test)

#display
intlabels = np.unique(y_test)

strlabels = []
for i in range(0, len(intlabels)):
    strlabels.append(binstr_to_name('{0:b}'.format(intlabels[i]).zfill(4)))
    print(strlabels[i])
pd.DataFrame(conmat, index=strlabels, columns=strlabels)

### Classes that do not exist in the dataset

In [ ]:
#"{0:b}".format()
y_unique = y_test.apply(lambda x: '{0:b}'.format(x).zfill(4)).unique()

s = []
for i in range(0,16):
    s.append('{0:b}'.format(i).zfill(4))
    
noclass = set(s) - set(y_unique)

d = ['INSULIN', 'METFORMIN', 'GLP1', 'SGLT2']
print('Non-existent combinations: ')
for i in noclass:
    noexists = []
    for j in range(0, len(i)):
        if i[j] == '1':
            noexists.append(d[j])
    if noexists:
        print('-', ' + '.join(noexists))